# RAG (Retrieval Augmented Generation) and Chatbot

## Pre-Requisites

### LLM model (VARCO) deploy in SageMaker
Make sure that you have ran the Notebook 1_deploy-varco_model_13_IST.ipynb

In [2]:
%store -r endpoint_name

In [3]:
import os
os.environ["VARCO_ENDPOINT"]=endpoint_name
print(os.environ["VARCO_ENDPOINT"])

varco-llm-13b-ist


In [4]:
!apt update

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.debian.org/debian-security buster/updates InRelease [34.8 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [551 kB]
Get:5 http://deb.debian.org/debian buster/main amd64 Packages [7909 kB]
Get:6 http://deb.debian.org/debian buster-updates/main amd64 Packages [8788 B]
Fetched 8682 kB in 2s (4197 kB/s)3m                  
Reading package lists... Done
Building dependency tree       
Reading state information... Done
65 packages can be upgraded. Run 'apt list --upgradable' to see them.


In [5]:
!apt install wkhtmltopdf -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  adwaita-icon-theme at-spi2-core avahi-daemon dbus-user-session
  dconf-gsettings-backend dconf-service dmsetup geoclue-2.0 glib-networking
  glib-networking-common glib-networking-services gsettings-desktop-schemas
  gstreamer1.0-plugins-base gtk-update-icon-cache iio-sensor-proxy iso-codes
  libargon2-1 libatk-bridge2.0-0 libatk1.0-0 libatk1.0-data libatspi2.0-0
  libavahi-core7 libavahi-glib1 libbrotli1 libcairo-gobject2 libcap2-bin
  libcdparanoia0 libcolord2 libcroco3 libcryptsetup12 libdaemon0 libdconf1
  libdevmapper1.02.1 libdouble-conversion1 libdrm-amdgpu1 libdrm-common
  libdrm-intel1 libdrm-nouveau2 libdrm-radeon1 libdrm2 libegl-mesa0 libegl1
  libelf1 libepoxy0 libevdev2 libfontenc1 libgbm1 libgdk-pixbuf2.0-0
  libgdk-pixbuf2.0-bin libgdk-pixbuf2.0-common libgl1 libgl1-mesa-dri
  libglapi-mesa libglvnd0 libglx-mesa0 libglx0 l

### Install certain libraries which are needed for this run.
These are provided in the requirements.txt or you can run these cells to fine control which libraries you need

In [6]:
!pip install --upgrade pip

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [7]:
!pip install langchain==0.0.161 --quiet

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [8]:
# !pip install chromadb==0.3.21 --quiet

In [9]:
!pip install langchain==0.0.161 boto3 html2text jinja2 --quiet

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [10]:
!pip install faiss-cpu==1.7.4 --quiet

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [11]:
!pip install pypdf==3.8.1 --quiet

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [12]:
!pip install transformers==4.24.0 --quiet

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [13]:
!pip install sentence_transformers==2.2.2 --quiet

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [14]:
!pip install pdfkit

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [15]:
import sentence_transformers 
sentence_transformers.__version__

'2.2.2'

In [16]:
print("all libraries installed")

all libraries installed


### Import statements for our chain and indexers. We are not using any explicit agent here

In [17]:
#from aws_langchain.kendra_index_retriever import KendraIndexRetriever
from langchain.chains import ConversationalRetrievalChain
from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import ContentHandlerBase
from langchain.prompts import PromptTemplate
import sys
import json
import os
import time
import sagemaker, boto3, json
from sagemaker.session import Session
from sagemaker.model import Model
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base
from typing import Any, Dict, List, Optional
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.llms.sagemaker_endpoint import ContentHandlerBase

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [18]:
import sagemaker
import boto3
import jinja2
role = sagemaker.get_execution_role()  # execution role for the endpoint

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


### [Optiona] Deploy a GPT-J embeddings Model - so we can use that to generate the embeddings for the documents

skip

### Use HuggingFaceEmbeddings in the workshop setting. 
If you are in a workshop, please use the below code. If you are using GPTJ model for generating the embeddings, please comment the below cell. 

In [19]:
from langchain.embeddings import HuggingFaceEmbeddings
from typing import Any, Dict, List, Optional
from pydantic import BaseModel, Extra, Field
from langchain.embeddings.base import Embeddings
import numpy as np

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}


class CustomHFEmbeddings(HuggingFaceEmbeddings):
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Compute doc embeddings using a HuggingFace transformer model.

        Args:
            texts: The list of texts to embed.

        Returns:
            List of embeddings, one for each text.
        """
        texts = list(map(lambda x: x.replace("\n", " "), texts))
        embeddings = self.client.encode(texts, **self.encode_kwargs)
        #- (22, 1536)
        print(f"CustomHFEmbeddings::embed_documents::shape:returned -- > {embeddings.shape}:")
        
        return embeddings.tolist()
    def embed_query(self, text: str) -> List[float]:
            """Compute query embeddings using a HuggingFace transformer model.

            Args:
                text: The text to embed.

            Returns:
                Embeddings for the text.
            """
            text = text.replace("\n", " ")
            embedding = self.client.encode(text, **self.encode_kwargs)
            print(f"CustomHFEmbeddings::QUERY::shape:returned -- > {embedding.shape}:")
            return embedding.tolist()

hf_embeddings = CustomHFEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

### Test the VARCO model 
Testing VARCO 13B IST model for answering a random question.

In [32]:
sagemaker_session = sagemaker.Session()
runtime = boto3.client("runtime.sagemaker")

model_name = "varco-llm-13b-ist"
content_type = "application/json"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [33]:
prompt = {"text":"Answer this question below, How can it help me?"}
print(f"Question being asked is -- > {prompt}:")

response = runtime.invoke_endpoint(
    EndpointName=model_name,
    ContentType=content_type,
    Accept="application/json",
    Body=json.dumps(prompt)
)

json.load(response["Body"])

Question being asked is -- > {'text': 'Answer this question below, How can it help me?'}:


{'result': ['### User:\nAnswer this question below, How can it help me?\n\n### Assistant:\nThe question is unclear. The answer is:\nHow can I help you?']}

## Section 2: Use LangChain

We will follow this pattern for the rest of the section

<li>Exploring vector databases
<li>Basics of QA exploring simple chains
<li>Basics of chatbot
<li>Going to prompt templates,
<li>Exploring Chains

### Exploring Vector DataBases and Create the Embeddings. 

Leverage SageMaker GPT-J model or the same

#### Use the file based document to retrieve based on embeddings

Run the below to visualize the Dataset

In [34]:
import glob
import os
import pandas as pd

all_files = glob.glob(os.path.join("rag_data/", "*.csv"))

df_knowledge = pd.concat(
    (pd.read_csv(f, header=None, names=["Question", "Answer"]) for f in all_files),
    axis=0,
    ignore_index=True,
)

#- drop 
df_answer = df_knowledge.drop(["Question"], axis=1)

print(df_knowledge.shape)
df_knowledge.head(2)

(154, 2)


,Question,Answer
0,Amazon SageMaker 란 무엇입니까?,"Amazon SageMaker는 완전 관리형 인프라, 도구 및 워크플로를 통해 모든..."
1,In which Regions is Amazon SageMaker available...,For a list of the supported Amazon SageMaker A...


#### Convert csv to pdf
temp.html 로컬 파일 생성

In [49]:
!pip install weasyprint

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [53]:
from bs4 import BeautifulSoup
import requests

# 한글 FAQ 문서 URL
url = 'https://aws.amazon.com/ko/sagemaker/faqs'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Add a style tag with desired font
font_tag = soup.new_tag('style')
font_tag.string = "@import url('https://fonts.googleapis.com/css2?family=Noto+Sans+KR&display=swap'); body { font-family: 'Noto Sans KR', sans-serif; }"
soup.head.append(font_tag)

# Save the modified HTML to a temporary file.
with open('rag_data/temp.html', 'w') as f:
    f.write(str(soup))


temp.html 파일을 Amazon_SageMaker_FAQs.pdf로 변환

In [59]:
# Generate the PDF from the modified HTML.

from weasyprint import HTML

HTML('rag_data/temp.html').write_pdf('rag_data/Amazon_SageMaker_FAQs.pdf')


AttributeError: 'NoneType' object has no attribute 'getBestCmap'


## <i> 한글 PDF 파일 만드는 로직은 조금 더 고민.. 일단은 SageMaker FAQ (한글) PDF 파일은 수동으로 생성하여 rag_data에 업로드

In [60]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma, AtlasDB, FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders.csv_loader import CSVLoader

In [61]:
import time
import sagemaker, boto3, json
from sagemaker.session import Session
from sagemaker.model import Model
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base
from typing import Any, Dict, List, Optional
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.llms.sagemaker_endpoint import ContentHandlerBase

#### Create the embeddings for document search

In [62]:
from langchain.indexes import VectorstoreIndexCreator

#### Vector store indexer. 

This is what stores and matches the embeddings.This notebook showcases Chroma and FAISS and will be transient and in memory. The VectorStore Api's are available [here](https://python.langchain.com/en/harrison-docs-refactor-3-24/reference/modules/vectorstore.html)

We will use our own Custom implementation of SageMaker Embeddings which needs a reference to the SageMaker endpoint to call the model which will return the embeddings. This will be used by the FAISS or Chroma to store in memory and be used when ever the User runs a query

#### Use LangChain to leverage a SageMaker LLM 

Let's break down the above VectorstoreIndexCreator and see what's happening under the hood. Furthermore, we will see how to incorporate a customize prompt rather than using a default prompt with VectorstoreIndexCreator.

Firstly, we generate embedings for each of document in the knowledge library with SageMaker  embedding model.


In [74]:
from langchain.llms.sagemaker_endpoint import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import ast

parameters = {
    "max_length": 200,
    "num_return_sequences": 1,
    "top_k": 250,
    "top_p": 0.95,
    "do_sample": False,
    "temperature": 1,
}
MAX_CHARACTER_TRUNCATION=10000 # at 20k it produced garbage results

class ContentHandlerSMLMI(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        #input_str = json.dumps({"text_inputs": prompt, **model_kwargs})
        print(f"ContentHandlerSMLMI::LangChain:::LEN:input_str={len(prompt)}:: will truncate if > {MAX_CHARACTER_TRUNCATION}::")
        if len(prompt) > MAX_CHARACTER_TRUNCATION:
            prompt=prompt[:MAX_CHARACTER_TRUNCATION]
        input_str = json.dumps({"text_inputs": prompt, **model_kwargs})
        #print(f"ContentHandlerSMLMI::LangChain:::LEN:input_str={len(input_str)}::")
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json_dict = json.loads(output.read().decode("utf-8"))
        print(f"ContentHandlerSMLMI::LangChain::output={response_json_dict}:")
        return response_json_dict[list(response_json_dict.keys())[0]] [0]


content_handler_sm_llm = ContentHandlerSMLMI()
session = boto3.Session()
boto3_sm_client = boto3.client(
    "sagemaker-runtime"
    # **boto3_kwargs
)
print(boto3_sm_client)


sm_llm = SagemakerEndpoint(
    client = boto3_sm_client,
    endpoint_name=os.environ["VARCO_ENDPOINT"],
    region_name='us-west-2',
    #model_kwargs=parameters,
    content_handler=content_handler_sm_llm,
)

print(f"SageMaker LLM created at {sm_llm}::")

SageMaker LLM created at SagemakerEndpoint
Params: {'endpoint_name': 'varco-llm-13b-ist', 'model_kwargs': {}}::


#### Load the Data from our Documents Source. 

Then we will feed this into the VectorStore to create the embeddings using the loaders like [here](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/directory_loader.html). First we will try with the SageMaker FAQ PDF document and also the IRS PDF files

we will create 3 Loaders and 3 documents after doing a split on them. 1st loader for amazon faq, 2nd for some of the IRS PDF's, 3rd just for  some ramdom example. For text it will be just a separate loader, text loader vs pdf

In [75]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders.csv_loader import CSVLoader

from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("rag_data/Amazon_SageMaker_FAQs.pdf")
documents_aws = loader.load() # -- gives 2 docs
documents_split = loader.load_and_split() # - gives 22 docs

vectorstore_faiss_aws = FAISS.from_documents(
    CharacterTextSplitter(chunk_size=300, chunk_overlap=0).split_documents(documents_aws), 
    hf_embeddings, 
    #k=1
    #**k_args
)#### VectorStore as FAISS 

You can read up about [FAISS](https://arxiv.org/pdf/1702.08734.pdf) in memory vector store here. However for our example it will be the same 

Chroma

[Chroma](https://www.trychroma.com/) is a super simple vector search database. The core-API consists of just four functions, allowing users to build an in-memory document-vector store. By default Chroma uses the Hugging Face transformers library to vectorize documents.

Weaviate

[Weaviate](https://github.com/weaviate/weaviate) is a very posh looking tool - not only does Weaviate offer a GraphQL API with support for vector search. It also allows users to vectorize their content using Weaviate's inbuilt modules or custom modules.

In [76]:
%%time
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import TextLoader
from langchain.document_loaders.csv_loader import CSVLoader

from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma, AtlasDB, FAISS

from langchain.document_loaders import PyPDFLoader
import glob
import os
import pandas as pd
from langchain.document_loaders import DirectoryLoader

from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

k_args = {"k": 1}
# - sub_docs = self.text_splitter.split_documents(docs)
# - create Vectorstore
vectorstore_faiss_aws = FAISS.from_documents(
    CharacterTextSplitter(chunk_size=300, chunk_overlap=0).split_documents(documents_aws), 
    hf_embeddings, 
    #k=1
    #**k_args
)

wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)

CustomHFEmbeddings::embed_documents::shape:returned -- > (18, 768):
CPU times: user 14.7 s, sys: 4.55 s, total: 19.3 s
Wall time: 23.3 s


#### First way of running the Query. High Level abstraction

Leverage VectorStoreIndexCreator which wraps around the RetrievalQA and provides a high level API abstraction to generate the response. This is a wrapper around the underlying API's which we will explore below

In [77]:
#query="Simplified method for business use of home deduction"
query="Amazon SageMaker 란 무엇인가"

In [78]:
wrapper_store_faiss.query(question="Amazon SageMaker ?",llm=sm_llm)

CustomHFEmbeddings::QUERY::shape:returned -- > (768,):
ContentHandlerSMLMI::LangChain:::LEN:input_str=10701:: will truncate if > 10000::


ValueError: Error raised by inference endpoint: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (422) from primary and could not load the entire response body. See https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/varco-llm-13b-ist in account 376278017302 for more information.